# Multipool Google Sentiment Analysis

this function will get data from google_result table
and calculate the sentiment analysis. Result is store in google_result_sentiment

crontab command
* * * * * /home/haviz/multipool/ai-case-study/run-ai-sa-google.sh >> /home/haviz/multipool/ai-sa-google.log



In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import pymysql.cursors
import sqlalchemy as sa
import nltk
import string
import os

from nltk.corpus import stopwords 
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker
from tqdm import tqdm
import pandas as pd

# nltk.download('stopwords')


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")

In [2]:
from sqlalchemy.util import deprecations
deprecations.SILENCE_UBER_WARNING = True

def execute_query(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '98.98.117.105',
        'port': '5432',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a metadata object
    metadata = MetaData()

    # Example: Define your table
    your_table = Table('your_table', metadata,
                       Column('id', Integer, primary_key=True),
                       Column('column1', String),
                       Column('column2', String)
                       )

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            # print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()

In [3]:
import pandas as pd
import re
from sqlalchemy import create_engine

def remove_mentions(tweet):
    # Define regular expression pattern to match mentions
    pattern = r'@\w+'
    
    # Replace mentions with an empty string
    cleaned_tweet = re.sub(pattern, '', tweet)
    
    return cleaned_tweet

def stemming(comment):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in comment:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    return d_clean
    
# function case folding
def casefolding(comment):
    comment = comment.lower()
    comment = comment.strip(" ")
    comment = re.sub(r'[?|$|.|!_:")(-+,]','',comment)
    return comment

def clean_up_tag(comment):
    x_ret = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",comment).split())
    return x_ret

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    
    # remove whitspace
    strOut = strOut.strip()
    
    # 
    strOut = re.sub('\s+',' ',strOut)
    return strOut

# Check for return value NaN or None
import numpy as np

def is_nan_value(value):
    """
    Check if the given value is NaN (Not a Number) or None.
    
    Parameters:
        value: Any - The value to check.
    
    Returns:
        bool: True if the value is NaN or None, False otherwise.
    """
    if isinstance(value, (int, float, np.number)):
        return np.isnan(value)
    elif value is None:
        return True
    else:
        return False


In [4]:
def process_by_df(source_ds):
    from tqdm import tqdm
    pd.options.mode.chained_assignment = None 

    # we calculate sentiment for 'title' and 'description' column

    # remove first
    source_ds['title'] = source_ds['title'].apply(clean_up_tag)
    source_ds['description'] = source_ds['description'].apply(clean_up_tag)


    # skip stemming
    source_ds['stemmed'] = source_ds['title']
    source_ds['stemmed'] = source_ds['stemmed'].apply(casefolding)
    source_ds['stemmed'] = source_ds['stemmed'].apply(text_preproc)

    sw = stopwords.words('indonesian')
    # tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay

    # Create a new list and insert each element from the original list
    from tqdm import tqdm

    list_text = source_ds['stemmed'].tolist()
    new_list = []
    for i in tqdm(range( len(list_text) )):
        res = pipe(list_text[i])
        #print(res)
        new_list.append(res)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay
    # create a list of our conditions
    for i in range(0, len(new_list)):
        # print(new_list[i][0]['label'])
        if new_list[i][0]['label'] == 'Positive':
            new_list[i][0]['Value'] = 1
        elif new_list[i][0]['label'] == 'Negative':
            new_list[i][0]['Value'] = -1
        elif new_list[i][0]['label'] == 'Neutral':
            new_list[i][0]['Value'] = 0

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']
        source_ds.at[index,'Score'] = new_list[index][0]['score']
        #source_ds.at[index,'Score'] = new_list[index][0]['Value']
        
    return source_ds


In [5]:
# Set up logging to a file
import logging
logging.basicConfig(filename='error.log', level=logging.ERROR)

def calculate_sentiment(row, col_description):
    try:
        # cleansing
        strSource = row[col_description]

        # cleansing
        strSource = clean_up_tag(strSource)
        strSource = casefolding(strSource)
        strSource = text_preproc(strSource)
        strSource = remove_mentions(strSource)

        # tokenized
        sToken = nltk.word_tokenize(strSource)

        # remove stopwords
        sw = stopwords.words('indonesian')

        filtered_words = [word for word in sToken if word.lower() not in sw]
        filtered_text = ' '.join(filtered_words)

        # Notes on return sentiment values
        # 1 >> positif >> stay
        # 2 >> negatif >> convert to -1
        # 0 >> netral >> stay
        result = pipe(strSource)

        if result[0]['label'] == 'Negative':
            sentimentClass = -1
        elif result[0]['label'] == 'Positive':
            sentimentClass = 1
        else:
            sentimentClass = 0

        score = result[0]['score']
        # handles NaN or non value
        if (is_nan_value(score) or is_nan_value(sentimentClass)):
            score = 0
            sentimentClass = 0
            
        # put to result    
        return pd.Series({'sentiment_category': sentimentClass, 'score': score})

    except Exception as e:
        
        # Log the exception to the error.log file
        logging.error(f"An error occurred: {e}")
        # Return default values or None for the columns
        return pd.Series({'sentiment_category': None, 'score': None})

    

In [6]:
#
# calculate sentiment for youtube video
#
def record_youtube_sentiment(row):
    ssql = """
    INSERT INTO public.youtube_result_sentiment 
    (id, sentiment_category, score, youtube_id) 
    VALUES(nextval('youtube_result_sentiment_id_seq'::regclass), %s, %s, '%s');
    """ %(row['sentiment_category'],row['score'],row['id'])
    #
    # print(ssql)
    execute_query(ssql)

select_query = '''
SELECT y.id, y.title
FROM youtube y
LEFT JOIN youtube_result_sentiment s ON y.id = s.youtube_id
WHERE s.youtube_id IS NULL;
'''

source_ds3 = execute_query(select_query)
id = ''

if len(source_ds3) != 0:
    for i in tqdm(range(len(source_ds3))):
    # for i in range(len(source_ds3)):
        row = source_ds3.iloc[i]
        
        id = row['id']
        row = calculate_sentiment(row, 'title')    

        # check value first
        if (is_nan_value(row['sentiment_category']) or is_nan_value(row['score'])):
            # store to rdbms
            ssql = """
    INSERT INTO public.youtube_result_sentiment 
    (id, sentiment_category, score, youtube_id) 
    VALUES(nextval('youtube_result_sentiment_id_seq'::regclass), %s, %s, '%s');
            """ %('0','0',id)
        else:
            # store to rdbms
            ssql = """
INSERT INTO public.youtube_result_sentiment 
(id, sentiment_category, score, youtube_id) 
VALUES(nextval('youtube_result_sentiment_id_seq'::regclass), %s, %s, '%s');
        """ %(row['sentiment_category'],row['score'],id)


        # print(ssql)
        execute_query(ssql)
        
        
    #tqdm.pandas(desc="Processing", total=len(source_ds3))
    #source_ds3[['sentiment_category','score']] = source_ds3.progress_apply(calculate_sentiment, args=('title',), axis=1)
    #source_ds3.apply(record_youtube_sentiment, axis=1)
    


In [10]:
# 
# calculate sentiment result for facebook post
# 

def record_tiktok_sentiment(row):
    # check for NaN
    if (is_nan_value(row['sentiment_category']) or is_nan_value(row['score'])):
        ssql = """
    INSERT INTO facebook_post_result_sentiment 
    (id, sentiment_category, score, facebook_id) 
    VALUES(nextval('facebook_post_result_sentiment_id_seq'::regclass), %s, %s, %s);
    """ %('0','0',row['id'])
    else:
        ssql = """
    INSERT INTO facebook_post_result_sentiment 
    (id, sentiment_category, score, facebook_id) 
    VALUES(nextval('facebook_post_result_sentiment_id_seq'::regclass), %s, %s, %s);
    """ %(row['sentiment_category'],row['score'],row['id'])
    #
    # print(ssql)
    execute_query(ssql)

select_query = """
SELECT 	a.id, a.description, a.name 
FROM 	facebook_post a
LEFT JOIN facebook_post_result_sentiment b ON a.id = b.facebook_id
WHERE b.facebook_id IS NULL
ORDER BY a.id desc limit 1000
"""

source_ds3 = execute_query(select_query)

if len(source_ds3) != 0:
    tqdm.pandas(desc="Calculating SA", total = len(source_ds3))
    source_ds3[['sentiment_category','score']] = source_ds3.progress_apply(calculate_sentiment, args=('description',), axis=1)
    
    #tqdm.pandas(desc="Recording SA", total=len(source_ds3))
    #source_ds3.progress_apply(record_tiktok_sentiment, axis=1)


Calculating SA: 100%|██████████| 1000/1000 [19:30<00:00,  1.17s/it]


In [8]:
source_ds3.to_pickle('source_ds3')